In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
from pyfvm.mesh  import *
from pyfvm.field import *
from pyfvm.xnum  import *
from pyfvm.integration import *
import pyfvm.modelphy.euler as euler
import pyfvm.modeldisc      as modeldisc
#
plt.rcParams['figure.dpi']  = 80
plt.rcParams['savefig.dpi'] = 100
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
plt.rcParams["animation.html"] = "html5" # for matplotlib 2.0 and below, converts to x264 using ffmpeg video codec

In [15]:
nx   = 200
vavg = -.2
vmag = .6
lx   = 3.

def fu(x):
    return vavg + vmag*( (x<1)+(x>1)*(x<1.+lx)*(1.+lx-x)/lx )

def fp(x): # gamma = 1.4
    return (1. + .2*fu(x))**7.  # satisfies C- invariant to make only C+ wave

def frho(x):
    return 1.4 * fp(x)**(1./1.4)

meshsim  = unimesh(ncell=nx,  length=10.)

model = euler.model()
bcL = { 'type': 'dirichlet', 'prim': [frho(0.),  fu(0.),  fp(0.)]}
bcR = { 'type': 'dirichlet', 'prim': [frho(10.), fu(10.), fp(10.)]}

rhs = modeldisc.fvm(model, meshsim, muscl(vanalbada), bcL=bcL, bcR=bcR)
solver = rk3ssp(meshsim, rhs)

# computation
#
nsol    = 120
endtime = 6.
cfl     = .5

xc    = meshsim.centers()
finit = rhs.fdata(model.prim2cons([ frho(xc), fu(xc), fp(xc) ])) # rho, u, p

start = time.clock()
fsol = solver.solve(finit, cfl, np.linspace(0., endtime, nsol+1))
cputime = time.clock()-start

print "cpu time computation (",solver.nit,"it) :",cputime,"s"
print "  %.2f µs/cell/it"%(cputime*1.e6/solver.nit/meshsim.ncell)


In [16]:
varname='velocity'
solgrid = [ fsol[i].phydata(varname) for i in range(nsol+1) ]
vmin, vmax = np.min(solgrid), np.max(solgrid)
# Figure / Plot of final results
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12,4))
ax1.set_ylabel(varname) ; ax1.set_ylim(vmin, vmax)
ax1.grid(linestyle='--', color='0.5')
line1, = fsol[-1].plot(varname, 'k-', axes=ax1)
ax2.set_ylabel('t') ; ax2.set_xlim(0., 10.)
#ax2.grid(linestyle='--', color='0.5')
ttime = [ fsol[i].time for i in range(nsol+1) ]
xx, xt = np.meshgrid(xc, ttime)
#error = [ np.sqrt(np.sum((fsol[i].phydata(varname)-mach_th)**2)/nx)/mach_th for i in range(nsol+1) ]
#print error
flood  = ax2.contour(xx, xt, solgrid, np.linspace(vmin, vmax, 30))
line2, = ax2.plot([0., 10.], [ttime[-1], ttime[-1]], 'k--')


In [14]:
import matplotlib.animation as anim
#
def animate(k):
    #i = min(k, nsol)
    fsol[k].set_plotdata(line1, varname)
    line2.set_data([0., 10.], [ttime[k], ttime[k]])
    return line1, line2

ani = anim.FuncAnimation(fig=fig, func=animate, frames=range(nsol+1), interval=100, blit=True)
ani
#from IPython.display import HTML
#HTML(ani.to_html5_video()) # if no rcparams